In [250]:
import cv2
import numpy as np
import math

In [251]:
class Cell:
    def __init__(self, cell_id, centroid, real_centroid, area):
        self.cell_id = 0
        self.centroid = centroid
        self.real_centroid = real_centroid
        self.area = area
        self.path = []
        self.speed = [0]

In [252]:
# def preprocessing(gray):
#     # Value of kernel
#     kernel = np.ones((4, 4), np.uint8)
#     # Erode the images
#     img_erosion = cv2.erode(gray, kernel, iterations = 1)
#     # Dilate the images
#     img_dilation = cv2.dilate(img_erosion, kernel, iterations = 1)
#     # Thresholding
#     ret, thresh = cv2.threshold(img_dilation, 170, 255, cv2.THRESH_BINARY)
#     # Gaussian blur
#     thresh = cv2.GaussianBlur(thresh, (3, 3), 0)
#     return thresh

In [253]:
def preprocessing(gray):
    # Value of kernel
    kernel = np.ones((4, 4), np.uint8)
    # Erode the images
    img_erosion = cv2.erode(gray, kernel, iterations = 1)
    # Dilate the images
    img_dilation = cv2.dilate(img_erosion, kernel, iterations = 1)
    # Thresholding
    thresh = cv2.adaptiveThreshold(cv2.subtract(255, img_dilation), 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 8)
    return 255 - thresh

In [254]:
def click_event(event, x, y, flags, param):
    global position
    if event == cv2.EVENT_LBUTTONDOWN:
        position = (x, y)

In [255]:
# Capture a set of images as a video
cap = cv2.VideoCapture('PhC-C2DL-PSC/test/t%03d.tif')

In [256]:
# First frame or not
first_frame = True
# Store the information of cells
cells = []
# Speed of video
fps = 10
# Frame size
size=(720, 576)
# The number of the frame
n = 0
# The position of mouse click
position = (0, 0)
# Video type
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# Produce a video
videoWriter = cv2.VideoWriter('output.avi', fourcc, fps, size)

In [257]:
# Open the captures
while(cap.isOpened()):
    # Read a capture
    success, frame = cap.read()
    if not success:
        break
    # Convert a image as grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Pre-processing
    thresh1 = preprocessing(gray)
    # Find contours
    image, contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Frame window
    cv2.namedWindow('frame')
    # Mouse call back
    cv2.setMouseCallback('frame', click_event)
    # The cell's id is same with the frame number
    cell_id = "%03d" % n
    # First frame or not
    if first_frame:
        for contour in contours:
            M = cv2.moments(contour)
            # Centroid of contour
            if M["m00"] == 0:
                centroid = (0, 0)
            else:
                centroid = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                real_centroid = (M["m10"] / M["m00"], M["m01"] / M["m00"])
            # Area of contour
            area = cv2.contourArea(contour)
            if area < 10 or area > 400:
                continue
            else:
                # Create cell's bounding boxes
                x, y, w, h = cv2.boundingRect(contour)
                # Cell's bounding boxes
                frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
            # Create a cell
            cell = Cell(cell_id, centroid, real_centroid, area)
            # Record the path
            cell.path.append(centroid)
            # Record this cell
            cells.append(cell)
#             # Speed of a cell
#             frame = cv2.putText(frame, str(cell.speed[-1]), cell.centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
        first_frame = False
    else:
        for contour in contours:
            # A new cell or not
            new_cell = True
            M = cv2.moments(contour)
            # Centroid of contour
            if M["m00"] == 0:
                centroid = (0, 0)
            else:
                centroid = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                real_centroid = (M["m10"] / M["m00"], M["m01"] / M["m00"])
#             print(centroid)
            # Area of contour
            area = cv2.contourArea(contour)
#             print(area)
            if area < 15 or area > 400:
                continue
            else:
                # Create cell's bounding boxes
                x, y, w, h = cv2.boundingRect(contour)
                # Cell's bounding boxes
                frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)
            # Extract cells one by one
            for cell in cells:
                # The Euclidean distances
                s_distance = math.sqrt(pow((cell.real_centroid[0] - real_centroid[0]), 2) + pow((cell.real_centroid[1] - real_centroid[1]), 2))
#                 print(cell.centroid, centroid)
                if s_distance < 5:
                    cell.cell_id = cell_id
                    cell.speed.append(s_distance)
                    if area > cell.area * 2:
                        x, y, w, h = cv2.boundingRect(contour)
                        frame = cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
                    if is_cell == 1 or position == cell.centroid:
                        position  = centroid
                        e_distance = sum(cell.speed)
                        n_distance = math.sqrt(pow((cell.path[0][0] - centroid[0]), 2) + pow((cell.path[0][1] - centroid[1]), 2))
                        if n_distance == 0:
                            ratio = 0
                        else:
                            ratio = e_distance / n_distance
                        inform = 'Speed = ' + str(round(s_distance, 2)) + '\nEuclidean distances = ' + str(round(e_distance, 2))\
                        + '\nNet distance = ' + str(round(n_distance, 2)) + '\nRatio = ' + str(round(ratio, 2))
                        x, y = position
                        # Display the information
                        for i, txt in enumerate(inform.split('\n')):
                            y += 20
                            cv2.putText(frame, txt, (x, y), cv2.FONT_HERSHEY_SIMPLEX,  .5, (0, 255, 255), 1)
                    if centroid != cell.path[-1]:
                        cell.path.append(centroid)
                        cell.centroid = centroid
                        cell.real_centroid = real_centroid
                    if area < cell.area:
                        cell.area = area
                    is_cell = cv2.pointPolygonTest(contour, position, False)
                    new_cell = False
            # Create new cells
            if new_cell:
                cell = Cell(cell_id, centroid, real_centroid, area)
                cell.path.append(centroid)
                cells.append(cell)
    # Draw the path of cell motion
    for cell in cells:
        if cell.cell_id != cell_id:
            continue
        for i in range(len(cell.path) - 1):
            cv2.line(frame, cell.path[i], cell.path[i + 1], (255,0, 0), 1) 
    # Show the results
    cv2.imshow('frame', frame)
    # output as a video
    videoWriter.write(frame)
    # A time interval of 100 milliseconds and input 'q' for quit
    if cv2.waitKey(100) & 0xFF == ord('q'):
        break
    n += 1
# Release everything if job is finished
cap.release()
# Write it as a video
videoWriter.release()
cv2.destroyAllWindows()